In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


# 1. 通过torchvision加载Fashion-MNIST数据集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_train = torchvision.datasets.FashionMNIST(root='./data/FashionMNIST', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.FashionMNIST(root='./data/FashionMNIST', train=False, download=True, transform=transform)

train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

# 2. 基于PyTorch构造一个不同层数的MLP
# 定义一个MLP类，它继承自nn.Module，这是一个基本的神经网络模块。
# 构建一个包含四个全连接层的神经网络，每个层后跟一个激活函数。第一层将28x28的图像展平为784个元素的向量，最后一层输出10个类别的概率分布。
class MLP(nn.Module):
    def __init__(self, activation):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 256),
            activation,
            nn.Linear(256, 128),
            activation,
            nn.Linear(128, 64),
            activation,
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the image
        x = self.layers(x)
        return x

# 3. 训练模型，尝试使用不同的激活函数和优化器
activations = {
    'ReLU': nn.ReLU(),
    'Sigmoid': nn.Sigmoid(),
    'Tanh': nn.Tanh()
}

optimizers = {
    'SGD': optim.SGD,
    'Adam': optim.Adam
}

# 4. 分析模型在测试集上的性能并可视化
# 定义一个字典activations，包含不同的激活函数（ReLU、Sigmoid、Tanh）。
# 定义一个字典optimizers，包含不同的优化器（SGD、Adam）。
# 对于每种激活函数和优化器的组合，创建一个模型实例，定义交叉熵损失函数，并初始化优化器。
# 进行指定数量的训练周期（epochs），在每个周期中，先进行训练阶段，然后是测试阶段。
# 在训练阶段，遍历训练数据集，计算损失，执行反向传播，并更新模型参数。
# 在测试阶段，遍历测试数据集，计算损失和准确率，但不更新模型参数。

activation_results = {}
for activation_name, activation in activations.items():
    for optimizer_name, optimizer_class in optimizers.items():
        model = MLP(activation)
        criterion = nn.CrossEntropyLoss()
        optimizer = optimizer_class(model.parameters(), lr=0.001)

        epochs = 10
        train_losses = []
        test_losses = []
        test_accuracies = []

        for epoch in range(epochs):
            # 训练阶段
            model.train()
            running_loss = 0.0
            for images, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)
            train_losses.append(train_loss)

            # 测试阶段
            model.eval()
            running_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    running_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            test_loss = running_loss / len(test_loader)
            test_losses.append(test_loss)
            test_accuracy = correct / total
            test_accuracies.append(test_accuracy)

        activation_results[(activation_name, optimizer_name)] = (train_losses, test_losses, test_accuracies)

# 5. 分析模型设计对性能的影响并可视化
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

for (activation_name, optimizer_name), (train_losses, test_losses, test_accuracies) in activation_results.items():
    axes[0].plot(train_losses, label=f'{activation_name}-{optimizer_name} (Train)')
    axes[0].plot(test_losses, label=f'{activation_name}-{optimizer_name} (Test)')
    axes[1].plot(test_accuracies, label=f'{activation_name}-{optimizer_name}')

axes[0].set_title('Model Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()

axes[1].set_title('Model Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()


Using downloaded and verified file: ./data/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz
Extracting ./data/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data/FashionMNIST\FashionMNIST\raw



100.0%


Extracting ./data/FashionMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data/FashionMNIST\FashionMNIST\raw



100.0%


Extracting ./data/FashionMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data/FashionMNIST\FashionMNIST\raw



100.0%


Extracting ./data/FashionMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST\FashionMNIST\raw

